# Imports

In [1]:
import pandas as pd
import numpy as np

from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor,ElasticNet,RidgeCV, LinearRegression
from sklearn.feature_selection import RFE,RFECV
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV, KFold, StratifiedShuffleSplit

import seaborn as sns
import matplotlib.pyplot as plt

from collections import Counter
import os
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
sns.set()

# Machine Learning

## Feature selection

In [62]:
def get_genes(X,y,estimators,num_genes_fold,num_genes_final,folds):
    genes_related = []
    
    for estimator in estimators:
        selector = RFECV(estimator, min_features_to_select=num_genes_fold, cv=folds, step=1)
        selector = selector.fit(X, y)
        ranking = selector.ranking_
        ranking_idx = ranking.argsort()[-num_genes_fold:][::-1]
        genes_related+=list(X.columns[ranking_idx])

    dic_count = Counter(genes_related)
    dic_count = {k: v for k, v in sorted(dic_count.items(), key=lambda item: item[1])}
    top_genes = list(dic_count.keys())[-num_genes_final:]
    return top_genes



## Classification

## Ajuste de hiperparâmetros

In [6]:
def get_hiperparams(X_train,X_test,y_train,y_test,kfold,param_grid, model,w,h,df,plot,annot):
    
    gs = GridSearchCV(estimator=model,
                      param_grid=param_grid,
                      cv=kfold)
    best_gs = gs.fit(X_train,y_train)
    n = len(param_grid)
    scores = best_gs.cv_results_['mean_test_score']
    scores = scores.reshape(len(scores),)
    
    if n == 1:
        label = list(param_grid.keys())[0]
        par = list(param_grid.values())[0]
        plt.plot(par,scores)
        plt.ylabel('F1-score')
        plt.xlabel(label)
        plt.show()
        
    if n == 2:
        
        label1,label2 = param_grid.keys()
        par1,par2 = param_grid.values()
        scores_df = pd.DataFrame(columns=par1,index=par2)
        for i in range(len(par1)*len(par2)):
            col = i % len(par1)
            row = i // len(par1)
            scores_df.iloc[row,col] = scores[i]

        scores_df = scores_df.astype(float)
        if df:
            print(scores_df)
        if plot:
            fig= plt.figure(figsize=(w,h))
            heatmap = sns.heatmap(scores_df,annot=annot)        
            plt.xlabel(label1)
            plt.ylabel(label2)
            plt.show()

    return best_gs.best_params_

## Classification 

In [75]:
n_splits = 4
test_size = 0.3 #Hp
num_genes_fold = 15
num_genes_final = 5
folds = 10
events = os.listdir('Dados/Eventos adversos')
metrics = pd.DataFrame()
metrics_ = []

n_folds = 4
test_size = 0.35

kfold = KFold(n_folds,shuffle=True,random_state=42)

max_depth = 20
depths = [i for i in range(1,max_depth+1)]
param_grid = {'max_depth':depths,
              'splitter':['best', 'random'],
              'criterion':['gini','entropy']
             }


for event in events:
    dados = pd.read_csv(f'Dados/Eventos adversos/{event}')
    
    X = dados.iloc[:,:-1]
    y = dados.iloc[:,-1]
    
    genes = get_genes(X,y,estimators,num_genes_fold,num_genes_final,folds)
    X = X[genes]

    sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=42)

    tree_score = 0.0

    for train_index, test_index in sss.split(X, y):    
        X_train,X_test,y_train,y_test = X.iloc[train_index],X.iloc[test_index],y.iloc[train_index],y.iloc[test_index]
        hp = get_hiperparams(X_train,X_test,y_train,y_test,kfold,param_grid,DecisionTreeClassifier(random_state=42,),10,6,False,False,False)
        criterion_opt,max_depth_opt,splitter_opt = hp.values()
        #Decision Trees
        tree = make_pipeline(StandardScaler(),
                            DecisionTreeClassifier(random_state=42,
                                                   criterion=criterion_opt,
                                                   max_depth=max_depth_opt,
                                                   splitter=splitter_opt))

        tree.fit(X_train,y_train)
        pred_tree = tree.predict(X_test)
        tree_score += f1_score(y_test,pred_tree,average='weighted',pos_label="Plano ID")

    tree_score /= float(n_splits)
    
    tree_score = round(tree_score,2)
    
    metrics_.append(tree_score)

#     print(f"F1-score for {event[:-4]}: Decision Trees: {tree_score}")

metrics['Decision_trees'] = metrics_
metrics

,Decision_trees
0,0.69
1,0.85
2,0.70
3,0.55
4,0.62
5,0.54
6,0.55
7,0.81


# Classification for # of events

In [84]:
dados = pd.read_csv('Dados/dados soma.csv')

X_soma = dados.iloc[:,:-1]
y_soma = dados.iloc[:,-1]

estimators = [SVR(kernel='linear'),
              SGDRegressor(),
              ElasticNet(),
              RidgeCV(),
              LinearRegression()]

genes = get_genes(X_soma,y_soma,estimators,num_genes_fold,num_genes_final,folds)
X_soma = X_soma[genes]

n_splits = 4
test_size = 0.3 
num_genes_fold = 15
num_genes_final = 8
folds = 10

n_folds = 4
test_size = 0.35

kfold = KFold(n_folds,shuffle=True,random_state=42)

max_depth = 20
depths = [i for i in range(1,max_depth+1)]
param_grid = {'max_depth':depths,
              'splitter':['best', 'random'],
              'criterion':['gini','entropy']
             }

sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=42)

tree_score = 0.0

for train_index, test_index in sss.split(X_soma, y_soma):    
    X_train,X_test,y_train,y_test = X_soma.iloc[train_index],X_soma.iloc[test_index],y_soma.iloc[train_index],y_soma.iloc[test_index]
    hp = get_hiperparams(X_train,X_test,y_train,y_test,kfold,param_grid,DecisionTreeClassifier(random_state=42,),10,6,False,False,False)
    criterion_opt,max_depth_opt,splitter_opt = hp.values()

    #Decision Trees
    tree = make_pipeline(StandardScaler(),
                        DecisionTreeClassifier(random_state=42,
                                               criterion=criterion_opt,
                                               max_depth=max_depth_opt,
                                               splitter=splitter_opt))

    tree.fit(X_train,y_train)
    pred_tree = tree.predict(X_test)
    tree_score += f1_score(y_test,pred_tree,average='weighted',pos_label="Plano ID")

tree_score /= float(n_splits)

tree_score = round(tree_score,2)


print(f"F1-score: Decision Trees: {tree_score*100}%")

F1-score: Decision Trees: 65.0%


In [72]:
y_train

323    2
83     1
75     1
281    1
60     1
      ..
245    1
267    1
395    2
379    2
107    2
Name: Class, Length: 282, dtype: int64